In [1]:
import pandas as pd
import numpy as np
from github import Auth
from github import Github
import json
import pickle
import os
import time
import shutil
from tqdm import tqdm

In [2]:
ACCESS_TOKEN = json.load(open("./config"))["access_token"]

In [3]:
import time

def rate_limited(requests_per_hour=5000):
    # Calculate the interval based on the rate limit (requests per hour)
    interval = 3600 / requests_per_hour  # interval in seconds
    last_executed = 0  # Track the last time a request was made

    def decorator(func):
        def wrapper(*args, **kwargs):
            nonlocal last_executed
            current_time = time.time()

            # If enough time has passed, execute the function
            if current_time - last_executed >= interval:
                last_executed = current_time
                return func(*args, **kwargs)
            else:
                # Sleep until the next allowed request time
                time.sleep(interval - (current_time - last_executed))
                last_executed = time.time()  # Update last execution time
                return func(*args, **kwargs)

        return wrapper
    return decorator


In [4]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import os

# Load database URL from environment variables or use default
DATABASE_URL = os.getenv("DATABASE_URL", "postgresql://bc:bc@localhost:5435/postgres")

# Create a database engine
engine = create_engine(DATABASE_URL)

# Base class for ORM models
Base = declarative_base()

# Create a session factory
SessionLocal = sessionmaker(bind=engine, expire_on_commit=False)

/tmp/ipykernel_2022985/84227372.py:13: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [5]:
from sqlalchemy import create_engine, Column, BigInteger, String, Integer, Boolean, TIMESTAMP, Text, ARRAY, func, \
    Sequence
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker


# Define the GitHub Repository ORM Model
class RepositoryBase(Base):
    __tablename__ = "repository"

    id = Column(BigInteger, Sequence('repository_id_seq', start=1, increment=1),
                primary_key=True)  # Auto-incrementing ID
    project_id = Column(BigInteger, unique=True)  # Unique GitHub Repository ID
    name = Column(String(255), nullable=False)  # Repository Name
    full_name = Column(String(255), nullable=False)  # Full name (owner/repo)
    is_fork = Column(Boolean, nullable=False, default=False)  # Fork status
    owner = Column(String(255), nullable=False)  # Repository Owner
    owner_url = Column(Text)  # Owner GitHub Profile URL
    repo_url = Column(Text, nullable=False)  # Repository GitHub URL
    stars = Column(Integer, nullable=False, default=0)  # Star Count
    forks = Column(Integer, nullable=False, default=0)  # Fork Count
    watchers = Column(Integer, nullable=False, default=0)  # Watchers Count
    language = Column(String(50))  # Primary Language
    description = Column(Text)  # Repository Description
    open_issues = Column(Integer, default=0)  # Open Issues Count
    license_name = Column(String(255))  # License Type
    topics = Column(ARRAY(Text))  # Array of Topics/Tags
    default_branch = Column(String(50))  # Default Branch (main/master)
    commit_hash = Column(String(255))  # Latest Commit Hash
    pushed_at = Column(TIMESTAMP)  # Last Commit Push Timestamp
    repository_created_at = Column(TIMESTAMP)  # Repository Creation Date
    repository_updated_at = Column(TIMESTAMP)  # Last Update Timestamp
    created_at = Column(TIMESTAMP, server_default=func.now())
    updated_at = Column(TIMESTAMP, server_default=func.now(), onupdate=func.now())


# Create table if not exists
Base.metadata.create_all(SessionLocal().bind)


# Repository CRUD Operations
class Repository:
    def __init__(self):
        self.session = SessionLocal()

    def add_repository(self, repo_data):
        """Insert a new repository"""
        new_repo = RepositoryBase(**repo_data)
        self.session.add(new_repo)
        self.session.commit()
        print(f"Added repository: {new_repo.name}")

    def get_repository(self, repo_id):
        """Fetch repository by ID"""
        return self.session.query(RepositoryBase).filter_by(id=repo_id).first()
    
    def get_repository_by_project_id(self, project_id):
        """Fetch repository by ID"""
        return self.session.query(RepositoryBase).filter_by(project_id=project_id).first()

    def update_repository(self, repo_id, update_data):
        """Update repository data"""
        repo = self.get_repository(repo_id)
        if repo:
            for key, value in update_data.items():
                setattr(repo, key, value)
            self.session.commit()
            print(f"Updated repository: {repo.name}")
        else:
            print("Repository not found.")

    def delete_repository(self, repo_id):
        """Delete repository by ID"""
        repo = self.get_repository(repo_id)
        if repo:
            self.session.delete(repo)
            self.session.commit()
            print(f"Deleted repository: {repo.name}")
        else:
            print("Repository not found.")

    def list_top_repositories(self, limit=1000):
        """Fetch top repositories by stars"""
        return self.session.query(RepositoryBase).order_by(RepositoryBase.id.asc()).limit(limit).all()

    def close_session(self):
        """Close the database session"""
        self.session.close()

In [6]:

def save_repositories_to_db(repos_list):
    """Convert API response and store data into PostgreSQL"""
    session = SessionLocal()
    repositoryDao = Repository()

    for repo in repos_list:
        repositoryEntity = repositoryDao.get_repository_by_project_id(repo["id"])
        if not repositoryEntity:
            repositoryEntity = RepositoryBase()


        repositoryEntity.project_id=repo["id"]
        repositoryEntity.name=repo["name"]
        repositoryEntity.full_name=repo["full_name"]
        repositoryEntity.is_fork=repo["fork"]
        repositoryEntity.owner=repo["owner"]["login"]
        repositoryEntity.owner_url=repo["owner"]["html_url"]
        repositoryEntity.repo_url=repo["html_url"]
        repositoryEntity.stars=repo["stargazers_count"]
        repositoryEntity.forks=repo["forks_count"]
        repositoryEntity.watchers=repo["watchers_count"]
        repositoryEntity.language=repo.get("language")
        repositoryEntity.description=repo.get("description")
        repositoryEntity.open_issues=repo.get("open_issues_count", 0)
        repositoryEntity.license_name=repo["license"]["name"] if repo.get("license") else None,
        repositoryEntity.topics=repo.get("topics", [])
        repositoryEntity.default_branch=repo.get("default_branch")
        repositoryEntity.pushed_at=repo.get("pushed_at")
        repositoryEntity.repository_created_at=repo.get("created_at")
        repositoryEntity.repository_updated_at=repo.get("updated_at")
        
        session.merge(repositoryEntity)  # Use merge to update if exists
        session.commit()
        print(f"Stored repository: {repositoryEntity.name}")

    
    session.close()


In [ ]:
import os

import requests

BASE_URL = "https://api.github.com/search/repositories"
HEADERS = {
    "Accept": "application/vnd.github.v3+json",
    "Authorization": f'token {ACCESS_TOKEN}'
}
MAX_REPOSITORY_PER_PAGE = 100
REPOSITORY_LIMIT = 1000 # Github allows to query the first 1K 
@rate_limited(requests_per_hour=5000)
def fetch_repository_by_topic(topic, pageIndex):
    """
    Fetch repositories for a given topic and pageIndex.

    Parameters:
    - topic (str): The topic to search repositories by.
    - pageIndex (int): The page number (0-indexed) to fetch repositories.

    Returns:
    - list: List of repositories fetched from GitHub.
    """
    params = {
        "q": f"topic:{topic}",
        "sort": "created",  # Sorting by 'created' to ensure deterministic results
        "order": "desc",    # Sorting in descending order (latest updated first)
        "per_page": MAX_REPOSITORY_PER_PAGE,    # Max results per page
        "page": pageIndex + 1  # GitHub API uses 1-based page indexing
    }

    response = requests.get(BASE_URL, headers=HEADERS, params=params)

    if response.status_code == 200:
        data = response.json()
        return data.get("items", [])
    else:
        raise Exception(f"GitHub API request failed with status code {response.status_code}: {response.text}")

def fetch_all_repository_by_topic(topic):
    """
    Fetch all repositories for a given topic by iterating over all pages.
    
    Parameters:
    - topic (str): The topic to search repositories by.
    
    Returns:
    - list: List of all repositories fetched from GitHub.
    """
    all_repos = []
    pageIndex = 0

    while True and pageIndex*MAX_REPOSITORY_PER_PAGE < REPOSITORY_LIMIT:
        repos = fetch_repository_by_topic(topic, pageIndex)
        if not repos:  # No more repositories available
            break
        all_repos.extend(repos)
        pageIndex += 1  # Move to the next page

    return all_repos

In [ ]:
topics = ["blockchain", "ethereum", "solidity", "web3", "cryptocurrency"]

for topic in topics:
    print(f"Fetching Repositories by topic: {topic}")
    repositories = fetch_all_repository_by_topic(topic)
    save_repositories_to_db(repositories)

Fetching Repositories by topic: blockchain
Stored repository: fuel-core
Stored repository: go-ethereum
Stored repository: shardeum
Stored repository: union
Stored repository: anoma
Stored repository: hey
Stored repository: bitcoinbook
Stored repository: solidity
Stored repository: Follow
Stored repository: ethereumbook
Stored repository: gun
Stored repository: linera-protocol
Stored repository: diem
Stored repository: fabric
Stored repository: truffle
Stored repository: solana
Stored repository: full-blockchain-solidity-course-js
Stored repository: WTF-Solidity
Stored repository: full-blockchain-solidity-course-py
Stored repository: chia-blockchain
Stored repository: DeFi-Developer-Road-Map
Stored repository: Qbot
Stored repository: monero
Stored repository: foundry
Stored repository: substrate
Stored repository: ethers.js
Stored repository: blockchain
Stored repository: go-ibax
Stored repository: lnd
Stored repository: tachyon
Stored repository: smart-contract-best-practices
Stored re